***
## Consolidacion de plantillas informe 057
***



se requiere instalar la siguiente libreria desde la cosola
pip install openpyxl
***

In [1]:
import pandas as pd
import os
import openpyxl
from datetime import datetime
from datetime import time
from datetime import date

In [2]:
if "03_INFORMES GENERADOS" not in os.listdir("."): 
    os.mkdir("03_INFORMES GENERADOS")  # Si la carpeta no existe se creax  
    os.mkdir("03_INFORMES GENERADOS/01_POR SUCURSAL")  # Si la carpeta no existe se creax  
    os.mkdir("03_INFORMES GENERADOS/02_POR CIUDAD")  # Si la carpeta no existe se creax

In [3]:
sucur=pd.read_csv('caf suc.csv')
ar057=pd.read_csv('02_CONSOLIDADO/057 CON/00_con057.csv', encoding='utf-8')
ar057=pd.merge(ar057,sucur)
#ar057.to_excel('informe 057 consolidado.xlsx', sheet_name='057 con') #Validacion del insumo

In [4]:
# Funcion de normalizacion de fechas
def etiempo(j):
        try:
            d=datetime.strptime(j, '%Y-%m-%d')
        except:
            d=datetime.strptime(j, '%d/%m/%Y')
        return(d)

In [5]:
ar057['MESDT']=ar057['Periodo'].apply(lambda x: etiempo(x)) # Normalizacion del tiempo 

In [6]:
# Funcion que entrega un dataframe resultante
#Funcion de consolidacion general  
def conval(xmes,archivo,ciudad):
        ar057a=ar057[(ar057['Ciudad'] == ciudad)&
                     (ar057['Fuente'] == archivo)&
                     (ar057['MMES']==xmes)]
        val1=int(ar057a[['Items sin marca pendiente']].sum())
    
        ar057a=ar057[(ar057['Ciudad']==ciudad)&
                     (ar057['Fuente']==archivo)&
                     (ar057['MMES']==xmes)]
        val2=int(ar057a[['Items dispensados']].sum())        
        try:
            pendiente = "{:.2%}".format((val2-val1)/val2)
        except:
            pendiente=0
        try:    
            popor= "{:.2%}".format(1-(val2-val1)/val2)
        except:
            popor=0
        matrix=[val2,val2-val1, pendiente,popor]        
        return(matrix)

In [7]:
####################################################################################################
###########################         Funcion de totalizacion          ###############################
####################################################################################################

def mtotal(pc,ciudad):    #Se carga una lista con los reportes requeridos deben ser 4

        mcuatro = pd.DataFrame(columns=('A', 'B', 'C', 'D'))
        mes=['ENE', 'FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEP', 'OCT', 'NOV', 'DIC']

        ################################################################
        #####        4 BLOQUES PRINCIPALES    0 , 12, 24 Y 36       ####
        ################################################################
        for f in pc:
            for d in mes:
                res=conval(d,f,ciudad)
                mcuatro.loc[len(mcuatro)]=res
        ################################################################
        # BLOQUE SUBTOTAL  48  TOTAL PC ST & PC_VS   
        ################################################################
        base=0
        for s in range(12):
            a = mcuatro['A'][base+s]+mcuatro['A'][base+s+12]
            b = mcuatro['B'][base+s]+mcuatro['B'][base+s+12]
            try:
                c = "{:.2%}".format(b/a)
            except:
                c=0
            try:
                d = "{:.2%}".format(1-b/a)
            except:
                d = 0
            mcuatro.loc[len(mcuatro)]=[a, b, c, d]
        ################################################################
        # BLOQUE SUBTOTAL 60 TOTAL PS ST & PS_VS  
        ################################################################
        base=24
        for s in range(12):
            a = mcuatro['A'][base+s]+mcuatro['A'][base+s+12]
            b = mcuatro['B'][base+s]+mcuatro['B'][base+s+12]
            try:
                c = "{:.2%}".format(b/a)
            except:
                c=0
            try:
                d = "{:.2%}".format(1-b/a)
            except:
                d = 0
            mcuatro.loc[len(mcuatro)]=[a, b, c, d]
        ################################################################
        # BLOQUE SUBTOTAL 72 TOTAL PC_ST + PS_ST   
        ################################################################
        base=0
        for s in range(12):
            a = mcuatro['A'][base+s]+mcuatro['A'][base+s+24]
            b = mcuatro['B'][base+s]+mcuatro['B'][base+s+24]
            try:
                c = "{:.2%}".format(b/a)
            except:
                c=0
            try:
                d = "{:.2%}".format(1-b/a)
            except:
                d = 0
            mcuatro.loc[len(mcuatro)]=[a, b, c, d]    
        ################################################################
        # BLOQUE SUBTOTAL 84 TOTAL PC_ST + PS_ST   
        ################################################################
        base=12
        for s in range(12):
            a = mcuatro['A'][base+s]+mcuatro['A'][base+s+24]
            b = mcuatro['B'][base+s]+mcuatro['B'][base+s+24]
            try:
                c = "{:.2%}".format(b/a)
            except:
                c=0
            try:
                d = "{:.2%}".format(1-b/a)
            except:
                d = 0
            mcuatro.loc[len(mcuatro)]=[a, b, c, d]    
        ################################################################
        # BLOQUE SUBTOTAL 96 TOTAL PC_ST + PS_ST   
        ################################################################
        base=72
        for s in range(12):
            a = mcuatro['A'][base+s]+mcuatro['A'][base+s+12]
            b = mcuatro['B'][base+s]+mcuatro['B'][base+s+12]
            try:
                c = "{:.2%}".format(b/a)
            except:
                c=0
            try:
                d = "{:.2%}".format(1-b/a)
            except:
                d = 0
            mcuatro.loc[len(mcuatro)]=[a, b, c, d]        
        return(mcuatro)        #se retorna una matriz

In [8]:
############################################################################################
######################           Funcin de llamado de matrices        ###################### 
############################################################################################
def libro057(ciudadb, narchivo):
        pc = ['01_ST_057_PC.csv', '01_VS_057_PC.csv', '02_ST_057_PS.csv', '02_VS_057_PS.csv']
        pc2= ['03_ST_057_NC.csv', '03_VS_057_NC.csv', '04_ST_057_NS.csv', '04_VS_057_NS.csv']

     #   ciudadb='BOGOTA'

        bloque1 = mtotal(pc,ciudadb)
        bloque2 = mtotal(pc2,ciudadb)

        wb = openpyxl.load_workbook('00_PLANTILLAS/Formato 057.xlsx')
        modif= wb['057 Formato']
        modif['B1'] = ciudadb

        #$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$#$
        cbloques = ['A','B','C','D']
        ordenc = [0, 12, 24, 36, 48, 60, 72, 84, 96]     # Orden en que estan los bloques
        ordenb = ['H8', 'N8', 'H27', 'N27', 'B8', 'B27', 'H46', 'N46', 'B46']

        for z in range(9):
            corex = ordenb[z]
            bloquex = ordenc[z]

            col = ord(corex[0])
            fil = int(corex[1:])

            colcon = ['A','B','C','D']
            for h in range(4):
                for j in range(12):
                    modif[str(chr(col+h)+str(j+fil))]=bloque1[colcon[h]][j+bloquex]

        ###############  bloque 2        
        cbloques = ['A','B','C','D']
        ordenc = [0, 12, 24, 36, 48, 60, 72, 84, 96]     # Orden en que estan los bloques
        ordenb = ['H8', 'N8', 'H27', 'N27', 'B8', 'B27', 'H46', 'N46', 'B46']

        for z in range(9):
            corex = ordenb[z]
            bloquex = ordenc[z]

            col = ord(corex[0])
            fil = int(corex[1:])+57

            colcon = ['A','B','C','D']
            for h in range(4):
                for j in range(12):
                    modif[str(chr(col+h)+str(j+fil))]=bloque2[colcon[h]][j+bloquex]

        #@@@@@@@@@@@@@@@@@@     impresion        
        #wb.save('03_INFORMES GENERADOS/02_POR CIUDAD/Formato 057_'+ciudadb+'.xlsx')
        wb.save(narchivo+'.xlsx') #el nombre del archivo es uno de los argumentos de la funcion

In [9]:
####################################################################################################
################                            X SUCURSALES                          ################## 
################             Carpeta de informe x fecha de generacion             ################## 
####################################################################################################

vector=ar057[['Sucursal', 'Ciudad']].drop_duplicates() #Listado
csucur=list(set(vector['Sucursal']))

narcs = "03_INFORMES GENERADOS/01_POR SUCURSAL/Ind 057 X Suc_ " + str(date.today())
os.mkdir(narcs)  # Carpeta principal
    
for a in  csucur:
    carps = narcs + "/Inf 057 Suc_" + a + "/"
    os.mkdir(carps)
    sdsd = vector[vector['Sucursal']==a] 
    for r in list(sdsd['Ciudad']):
        libro057(r,carps +'Ind 057_' +r)        

In [12]:
####################################################################################################
################                         X TOTAS LAS CIUDADES                     ################## 
################             Carpeta de informe x fecha de generacion             ################## 
####################################################################################################

narcc = "03_INFORMES GENERADOS/02_POR CIUDAD/Ind 057 TOTAS LAS CIUDADES_ " + str(date.today())
os.mkdir(narcc)  # Carpeta principal
for r in list(set(vector['Ciudad'])):
    libro057(r,narcs +'/Ind 057_' + r) 